In [12]:
import re
import spacy
from spacy.matcher import Matcher
from datetime import datetime


def extract_info_from_resume(text):
    # 1. Extract name

    # 加载spacy的中文模型
    nlp = spacy.load('zh_core_web_sm')
    # 初始化匹配器
    matcher = Matcher(nlp.vocab)

    # 百家姓列表
    surnames = "赵钱孙李周吴郑王冯陈褚卫蒋沈韩杨朱秦尤许何吕施张孔曹严华金魏陶姜戚谢邹喻柏水窦章云苏潘葛奚范彭郎林鲁韦昌马苗凤"#乐于时傅皮卞齐康伍余元卜顾孟平黄和穆萧尹姚邵湛汪祁毛禹狄米贝明臧计伏成戴谈宋茅庞熊纪舒屈项祝董梁杜阮蓝闵席季麻强贾路娄危江童颜郭梅盛林刁钟徐邱骆高夏蔡田樊胡凌霍虞万支柯昝管卢莫经房裘缪干解应宗丁宣贲邓郁单杭洪包诸左石崔吉钮龚程嵇邢滑裴陆荣翁"

    # 创建姓名匹配模式
    pattern = [{"TEXT": {"REGEX": "[\u4e00-\u9fa5]{2,3}"}}]
    matcher.add("CHINESE_NAME", [pattern])

    doc = nlp(text)
    matches = matcher(doc)
    names = [doc[start:end].text for match_id, start, end in matches]
    print(names)
    if names:
        # 只选择以百家姓开头的名字
        valid_names = [name for name in names if name[0] in surnames]
        name = valid_names[0] if valid_names else None
    else:
        name = None

    # 2. Extract age
    birthdate_search = re.search(r"出生年月：(\d{4}.\d{1,2})", text)
    birthdate = birthdate_search.group(1) if birthdate_search else None
    age = None
    if birthdate:
        birthyear = int(birthdate.split(".")[0])
        current_year = datetime.now().year
        age = current_year - birthyear

    # 3. Extract education level
    education_search = re.search(r"学\s*历：(\w+)", text)
    education = education_search.group(1) if education_search else None

    # 4. Extract university
    university_search = re.search(r"(\d{4}.\d{1,2}-\d{4}.\d{1,2}\s+)([\u4E00-\u9FA5]+)(\s+[\u4E00-\u9FA5]+)", text)
    university = university_search.group(2) if university_search else None

    # 5. Extract years of experience
    experience_search = re.findall(r"(\d{4}.\d{1,2}-\d{4}.\d{1,2}|至今)", text)
    experience = len(experience_search) if experience_search else None

    return {"Name": name, "Age": age, "Education": education, "University": university, "Experience": experience}

# Read text from a file
with open("2.txt", "r", encoding="utf-8") as file:
    text = file.read()

print(extract_info_from_resume(text))


['求职', '意向', '市场', '专员', '中国', '传媒', '大学', '艺术', '设计', '教育', '背景', '腾讯', '公司', '负责', '公司', '团队', '组织', '建设', '团队', '精神', '培养', '协调', '部位', '工作', '组织', '策划', '团队', '成员', '大会', '竟选会', '团队', '培训', '招募', '成员', '工作', '市场', '经理', '阿里巴巴', '公司', '至今', '市场', '经理', '助理', '工作', '经历', '林国瑞', '语言', '普通话', '英语', '上海', '个人', '荣誉', '大学', '一直', '担任', '班长', '荣获', '优秀', '校园', '创业者', '协会', '担任', '宣传', '部长', '荣获', '第四', '全国', '大学生', '广告', '艺术', '设计', '大赛', '平面', '作品', '三等奖', '主修', '课程', '英文', '口语', '笔译', '口译', '英文', '写作', '文学', '导游', '实务', '商务', '礼仪', '文化', '交流', '旅游', '心理学', '自我', '学习', '能力', '还是', '比较', '认真', '专业', '知识', '扎实', '积极', '工作', '态度', '能够', '独立', '工作', '团队', '精神', '具有', '良好', '文化', '素质', '未来', '工作', '充沛', '精力', '努力', '工作', '稳定', '进步', '自己', '工作', '能力', '正在', '寻找', '一个', '好的', '发展', '平台', '希望', '能够', '充分', '发挥', '自己', '优势', '共同', '努力', '成就', '一番', '事业', '个人', '介绍', '掌握', '技能', '联系', '方式', '北京', '海淀区', '出生', '年月', '北京', '设计', '本科', '软件', '平面', '设计', '办公', '软件', '个人', '资料']
{'Name': '林国瑞', 'Age': 

In [1]:
import re
import jieba.posseg as pseg
from datetime import datetime

def extract_info_from_resume(text):
    # 1. Extract name
    # 百家姓列表
    surnames = "赵钱孙李周吴郑王冯陈褚卫蒋沈韩杨朱秦尤许何吕施张孔曹严华金魏陶姜戚谢邹喻柏水窦章云苏潘葛奚范彭郎鲁韦昌马苗凤花方俞任袁柳酆鲍史唐费廉岑薛雷贺倪汤滕殷罗毕郝邬安常乐于时傅皮卞齐康伍余元卜顾孟平黄和穆萧尹姚邵湛汪祁毛禹狄米贝明臧计伏成戴谈宋茅庞熊纪舒屈项祝董梁杜阮蓝闵席季麻强贾路娄危江童颜郭梅盛林刁钟徐邱骆高夏蔡田樊胡凌霍虞万支柏昝管卢莫经房裘缪干解应宗丁宣贲邓郁单杭洪包诸左石崔吉钮龚程嵇邢滑裴陆荣翁成魏连查乌苗凤花方俞任袁柳"

    words = pseg.cut(text)
    names = [word for word, flag in words if flag == 'nr']
    print(names)
    if names:
        # 只选择以百家姓开头的名字
        valid_names = [name for name in names if name[0] in surnames]
        name = valid_names[0] if valid_names else None
    else:
        name = None

    # 2. Extract age
    birthdate_search = re.search(r"出生年月：(\d{4}.\d{1,2})", text)
    birthdate = birthdate_search.group(1) if birthdate_search else None
    age = None
    if birthdate:
        birthyear = int(birthdate.split(".")[0])
        current_year = datetime.now().year
        age = current_year - birthyear

    # 3. Extract education level
    education_search = re.search(r"学\s*历：(\w+)", text)
    education = education_search.group(1) if education_search else None

    # 4. Extract university
    university_search = re.search(r"(\d{4}.\d{1,2}-\d{4}.\d{1,2}\s+)([\u4E00-\u9FA5]+)(\s+[\u4E00-\u9FA5]+)", text)
    university = university_search.group(2) if university_search else None

    # 5. Extract years of experience
    experience_search = re.findall(r"(\d{4}.\d{1,2}-\d{4}.\d{1,2}|至今)", text)
    experience = len(experience_search) if experience_search else None

    return {"Name": name, "Age": age, "Education": education, "University": university, "Experience": experience}

# Read text from a file
with open("2.txt", "r", encoding="utf-8") as file:
    text = file.read()

print(extract_info_from_resume(text))


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HUAWEI\AppData\Local\Temp\jieba.cache
Loading model cost 0.597 seconds.
Prefix dict has been built successfully.


['林国瑞', '荣誉', '荣获', '班干', '荣获', '贯']
{'Name': '林国瑞', 'Age': 33, 'Education': '本科', 'University': '中国传媒大学', 'Experience': 3}


In [2]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HUAWEI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import re
import jieba.posseg as pseg
from datetime import datetime
import math
from dateutil.relativedelta import relativedelta



def extract_info_from_resume(text):
    # 1. Extract name
    # 百家姓列表
    surnames = "赵钱孙李周吴郑王冯陈褚卫蒋沈韩杨朱秦尤许何吕施张孔曹严华金魏陶姜戚谢邹喻柏水窦章云苏潘葛奚范彭郎鲁韦昌马苗凤花方俞任袁柳酆鲍史唐费廉岑薛雷贺倪汤滕殷罗毕郝邬安常乐于时傅皮卞齐康伍余元卜顾孟平黄和穆萧尹姚邵湛汪祁毛禹狄米贝明臧计伏成戴谈宋茅庞熊纪舒屈项祝董梁杜阮蓝闵席季麻强贾路娄危江童颜郭梅盛林刁钟徐邱骆高夏蔡田樊胡凌霍虞万支柏昝管卢莫经房裘缪干解应宗丁宣贲邓郁单杭洪包诸左石崔吉钮龚程嵇邢滑裴陆荣翁成魏连查乌苗凤花方俞任袁柳"

    words = pseg.cut(text)
    names = [word for word, flag in words if flag == 'nr']
    if names:
        # 只选择以百家姓开头的名字
        valid_names = [name for name in names if name[0] in surnames]
        name = valid_names[0] if valid_names else None
    else:
        name = None

     # 2. Extract age
    birthdate_search = re.search(r"出生年月：(\d{4})", text)
    birthdate = birthdate_search.group(1) if birthdate_search else None
    age = None
    if birthdate:
        birthyear = int(birthdate)
        current_year = datetime.now().year
        age = current_year - birthyear + 1

    # 3. Extract education level
    education_search = re.search(r"学\s*历：(\w+)", text)
    education = education_search.group(1) if education_search else None



    # 4. Extract university
    
    with open("college.txt", 'r', encoding='utf-8') as f:
        colleges = f.read().splitlines()
        

    # 然后提取大学信息
    university_matches = [match for match in re.findall(r"([\u4E00-\u9FA5]+(?:大学|学院|学校|中专|职业技术学院|职业学院|高等专科学校|职业学校))", text) if match in colleges]

    if not university_matches:
        university = None
    else:
        university = university_matches[-1]




    # 5. Extract years of experience
    # 1. Replace "至今" with "present" to make processing easier
    text = text.replace("至今", "present")
    
    # 2. Find the index of the last education experience
    last_edu_index = 0
    for edu_match in re.finditer(r"\d{4}\.\d{1,2}-\d{4}\.\d{1,2}", text):
        last_edu_index = edu_match.end()

    # 3. Extract years of experience
    experience_search = re.findall(r"(\d{4}\.\d{1,2}-(?:\d{4}\.\d{1,2}|present))", text)
    total_months = 0

    for exp in experience_search:
        start_date, end_date = re.split(r'-', exp)
        start_year, start_month = map(int, start_date.split("."))
        if "present" in end_date:
            end_year, end_month = 2023, 5  # If "present", take as May 2023
        else:
            end_year, end_month = map(int, end_date.split("."))
        
        start_date = datetime(start_year, start_month, 1)
        end_date = datetime(end_year, end_month, 1)
        
        delta = relativedelta(end_date, start_date)
        total_months += delta.years*12 + delta.months

    # Experience in years, rounding up
    experience = math.ceil(total_months / 12)

    print(experience_search)


    return {"Name": name, "Age": age, "Education": education, "University": university, "Experience": experience}

# Read text from a file
with open("2.txt", "r", encoding="utf-8") as file:
    text = file.read()

print(extract_info_from_resume(text))


['2008.9-2012.6', '2013.1-2018.5', '2018.5-present']
{'Name': '林国瑞', 'Age': 34, 'Education': '本科', 'University': '中国传媒大学', 'Experience': 15}
